In [1]:
# First we will import neccessary libraries
import pandas as pd
import numpy as np
import tensorflow as tf
import os
import cv2 as cv
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.layers import Lambda
from tqdm import tqdm
import albumentations as A 
import shutil
import skimage
from skimage.transform import  resize

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [2]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Path to the dataset directory
dataset_dir = '/kaggle/input/dangerous-insects-dataset/farm_insects'

# Output directory for train and test sets
output_dir = '/kaggle/working/dataset'
os.makedirs(output_dir, exist_ok=True)

# Splitting the dataset into train and test sets
train_dir = os.path.join(output_dir, 'train')
test_dir = os.path.join(output_dir, 'test')
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Splitting the dataset into train and test sets
for class_name in os.listdir(dataset_dir):
    class_dir = os.path.join(dataset_dir, class_name)
    images = os.listdir(class_dir)
    
    # Split images into train and test sets
    train_images, test_images = train_test_split(images, test_size=0.3, random_state=42)
    
    # Create class directories in train and test sets
    class_train_dir = os.path.join(train_dir, class_name)
    class_test_dir = os.path.join(test_dir, class_name)
    os.makedirs(class_train_dir, exist_ok=True)
    os.makedirs(class_test_dir, exist_ok=True)
    
    # Copy images to their respective directories
    for image in train_images:
        src = os.path.join(class_dir, image)
        dst = os.path.join(class_train_dir, image)
        shutil.copyfile(src, dst)
        
    for image in test_images:
        src = os.path.join(class_dir, image)
        dst = os.path.join(class_test_dir, image)
        shutil.copyfile(src, dst)


In [3]:
sdir=('/kaggle/working/dataset/train')
working_dir=('/kaggle/working')
dest_dir=os.path.join(working_dir, 'balanced dataset')
print(f'the balanced dataset will be saved to {dest_dir}')
if os.path.isdir(dest_dir):
    shutil.rmtree(dest_dir) # start with a clean empty dest_dir
os.mkdir(dest_dir) 
limiter=int(220)
print(f'the dataset will be balanced to have {limiter} image files in each class')
print(f'processing files in train directory')
classes=sorted(os.listdir(sdir))# make a list of the classes
class_count=len(classes)# number of classes
bad_img_list=[]
ht=0
wt=0
good_file_count=0
for klass in classes: # iterate through the classes
    classpath=os.path.join(sdir,klass)
    dest_classpath=os.path.join(dest_dir, klass)
    os.mkdir(dest_classpath) # make a directory in the dest_dir for this class
    flist=sorted(os.listdir(classpath))# get list of files in each class    
    fcount=len(flist) 
    if fcount>limiter: # if the number of files is greator than the limiter sample the file list to have limiter number of files
        flist=np.random.choice(flist, limiter, replace=False)      
    desc=f'{klass[:25]:25s}' # show which class is being procesed
    for f in tqdm(flist, ncols=120, colour='blue', desc=desc, unit='files'): # iterate through the files in each class
        fpath=os.path.join(classpath,f)
        dest_fpath=os.path.join(dest_classpath, f) # where to store the file
        try: # test if the image file can be read if so save it to the dest_dir
            img= skimage.io.imread(fpath)            
            shape=img.shape 
            ht += shape[0]
            wt += shape[1]
            good_file_count +=1
            shutil.copy(fpath,  dest_fpath)  
        except: # if the image file can not be read store the path in the bad_img_list
            bad_img_list.append(fpath)
if len(bad_img_list)>0:
    print(f'the image files listed below could not be processed and were not included in the balanced dataset')
    for f in bad_img_list:
        print (f)
have=int(ht/good_file_count)
wave=int(wt/good_file_count)
print(f'the average height of the images= {have} the average_width of the images = {wave}')
# some files in the original dataset may have been invalid image files and were not copied to the destination directory
# so go through the destination directory and determine how many files are in each class and store in fcount_list

classes=sorted(os.listdir(dest_dir))
fcount_list=[]# list to store how many files are in each class
for klass in classes:
    classpath=os.path.join(dest_dir, klass)
    flist=sorted(os.listdir(classpath))
    file_count=len(flist)
    fcount_list.append(file_count)
zip_list=zip(fcount_list,classes)
ordered_list=sorted(zip_list) # order the list by the number of files
fc='File Count'
k='Class'
print(f'the dataset has {class_count} classes listed below:')
print(f'{fc:^12s}{k:^45s}')
classes=[]
fcount=[]
for count, klass in ordered_list:
    fcount.append(count)
    classes.append(klass)
    strcount=str(count)
    print(f'{strcount:^12s}{klass:^45s}')
min_class=classes[0]
min_count=fcount[0]
length=len(classes)-1
max_class=classes[length]
max_count=fcount[length]
print(f'class {max_class} has the most image files= {max_count}, class {min_class} has the least images= {min_count}')
# lets define a function that creates an augmented image

def get_augmented_image(image): # given an image this function returns an augmented image
    width=int(image.shape[1]*.8)
    height=int(image.shape[0]*.8)
    transform= A.Compose([
        A.HorizontalFlip(p=.5),
        A.Rotate(limit=30, p=.25),
        A.RandomBrightnessContrast(p=.5),
        A.RandomGamma(p=.5),
        A.RandomCrop(width=width, height=height, p=.25) ])    
    return transform(image=image)['image']

# Iterate through the classes, for a class with less than limiter number image files create augmented images
# so each class has limiter number of image files
no_save_list=[]
dummy_path=os.path.join(working_dir, 'dummy')
if os.path.isdir(dummy_path):
    shutil.rmtree(dummy_path)
os.mkdir(dummy_path)
for count, klass in ordered_list:
    aug_counter=0
    i= -1
    if count < limiter: # if this class has less than the limiter value of files
        delta=limiter - count # this is how many augmented images we need to create for this class
        print (f'creating {delta} augmented images for class {klass}')
        classpath=os.path.join(dest_dir,klass)# path to class
        dest_classpath=os.path.join(dest_dir,klass) # where to store files for this class  
        dummy_classpath=os.path.join(dummy_path, klass)
        os.mkdir(dummy_classpath)
        flist=sorted(os.listdir(classpath))# get list of files for this class         
        while aug_counter<delta:
            i +=1 
            j=i % count # delta may be greator than the number of files in the class so use mod of count as file indexer
            fpath=os.path.join(classpath, flist[j]) # path to the image file to augment
            
            # give the augmented image a unique file name
            dest_fpath=os.path.join(dest_classpath,'aug-'+str(i)+'-'+ flist[j])# path of where to store the augmented image
            dummy_fpath=os.path.join(dummy_classpath,'aug-'+str(i)+'-'+ flist[j])
            try:                
                img= skimage.io.imread(fpath)
                aug_img=get_augmented_image(img) # augment the image 
                skimage.io.imsave(dummy_fpath, aug_img)
                skimage.io.imsave(dest_fpath, aug_img) # save the augmented image
                aug_counter +=1
            except:
                no_save_list.append(fpath)
print(f'the balanced dataset with {limiter} image files in each class is stored in {dest_dir} ') 
shutil.rmtree(dummy_path)


the balanced dataset will be saved to /kaggle/working/balanced dataset
the dataset will be balanced to have 220 image files in each class
processing files in train directory


Brown Marmorated Stink Bu:  14%|███████▍                                             | 11/79 [00:00<00:00, 95.48files/s]/opt/conda/lib/python3.10/site-packages/PIL/Image.py:992: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
Western Corn Rootworms   : 100%|█████████████████████████████████████████████████████| 70/70 [00:00<00:00, 73.11files/s]


the average height of the images= 762 the average_width of the images = 1021
the dataset has 15 classes listed below:
 File Count                     Class                    
     61                        Aphids                    
     67         Africanized Honey Bees (Killer Bees)     
     67                       Armyworms                  
     70                      Fruit Flies                 
     70                Western Corn Rootworms            
     72                    Cabbage Loopers               
     72                     Citrus Canker                
     76                        Thrips                    
     76                   Tomato Hornworms               
     77                     Corn Earworms                
     78                Colorado Potato Beetles           
     79              Brown Marmorated Stink Bugs         
     79                    Fall Armyworms                
     80                      Corn Borers                 
     83     

/tmp/ipykernel_23/877597905.py:116: UserWarning: /kaggle/working/dummy/Fruit Flies/aug-83-Image_12.jpg is a low contrast image
  skimage.io.imsave(dummy_fpath, aug_img)


creating 150 augmented images for class Western Corn Rootworms
creating 148 augmented images for class Cabbage Loopers
creating 148 augmented images for class Citrus Canker
creating 144 augmented images for class Thrips
creating 144 augmented images for class Tomato Hornworms
creating 143 augmented images for class Corn Earworms
creating 142 augmented images for class Colorado Potato Beetles
creating 141 augmented images for class Brown Marmorated Stink Bugs
creating 141 augmented images for class Fall Armyworms
creating 140 augmented images for class Corn Borers
creating 137 augmented images for class Spider Mites
the balanced dataset with 220 image files in each class is stored in /kaggle/working/balanced dataset 


In [4]:
path = "/kaggle/working/balanced dataset"
key_value= dict()
for i in os.listdir(path):
    key_value[i] = os.listdir(path+'/'+i)
    

In [5]:
df = []
for key in key_value.keys():
    for j in key_value[key]:
        df.append([key,path+'/'+key+'/'+j])
df1 = pd.DataFrame(df,columns=('label','path'))
df1

,label,path
0,Cabbage Loopers,/kaggle/working/balanced dataset/Cabbage Loope...
1,Cabbage Loopers,/kaggle/working/balanced dataset/Cabbage Loope...
2,Cabbage Loopers,/kaggle/working/balanced dataset/Cabbage Loope...
3,Cabbage Loopers,/kaggle/working/balanced dataset/Cabbage Loope...
4,Cabbage Loopers,/kaggle/working/balanced dataset/Cabbage Loope...
...,...,...
3295,Corn Earworms,/kaggle/working/balanced dataset/Corn Earworms...
3296,Corn Earworms,/kaggle/working/balanced dataset/Corn Earworms...
3297,Corn Earworms,/kaggle/working/balanced dataset/Corn Earworms...
3298,Corn Earworms,/kaggle/working/balanced dataset/Corn Earworms...


In [6]:
df1['path']

0       /kaggle/working/balanced dataset/Cabbage Loope...
1       /kaggle/working/balanced dataset/Cabbage Loope...
2       /kaggle/working/balanced dataset/Cabbage Loope...
3       /kaggle/working/balanced dataset/Cabbage Loope...
4       /kaggle/working/balanced dataset/Cabbage Loope...
                              ...                        
3295    /kaggle/working/balanced dataset/Corn Earworms...
3296    /kaggle/working/balanced dataset/Corn Earworms...
3297    /kaggle/working/balanced dataset/Corn Earworms...
3298    /kaggle/working/balanced dataset/Corn Earworms...
3299    /kaggle/working/balanced dataset/Corn Earworms...
Name: path, Length: 3300, dtype: object

In [7]:
import cv2
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

images = []
labels = []

for path, label in zip(df1['path'], df1['label']):
    try:
        img = cv2.imread(path)
        img = cv2.resize(img, (250, 200))
        #img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
        #img = img.resize((300, 300 * img.shape[1] // img.shape[0]), Image.ANTIALIAS)
        if img is not None:
            images.append(img)
            labels.append(label)
        else:
            print(f"Image at path {path} is None")
    except Exception as e:
        print(f"Error reading or resizing image at path: {path}")
        print(f"Error message: {str(e)}")

train_images = np.array(images, dtype='int')
labels = np.array(labels)

le = preprocessing.LabelEncoder()
train_labels = le.fit_transform(labels)

# Split the data into training and testing
#train_images, test_images, train_labels, test_labels = train_test_split(images, y_labels, test_size=0.3, random_state=42)



Error reading or resizing image at path: /kaggle/working/balanced dataset/Western Corn Rootworms/aug-112-Image_64.gif
Error message: OpenCV(4.7.0) /io/opencv/modules/imgproc/src/resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Error reading or resizing image at path: /kaggle/working/balanced dataset/Western Corn Rootworms/Image_35.gif
Error message: OpenCV(4.7.0) /io/opencv/modules/imgproc/src/resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Error reading or resizing image at path: /kaggle/working/balanced dataset/Western Corn Rootworms/aug-91-Image_35.gif
Error message: OpenCV(4.7.0) /io/opencv/modules/imgproc/src/resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Error reading or resizing image at path: /kaggle/working/balanced dataset/Western Corn Rootworms/Image_64.gif
Error message: OpenCV(4.7.0) /io/opencv/modules/imgproc/src/resize.cpp:4062: error: (-215:Assertion failed) !s

In [8]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Increase input image size
base_model = keras.applications.ResNet50(
    weights='imagenet', include_top=False, input_shape=(200, 250, 3)
)
fine_tune_at = 150

# Adjust fine-tuning strategy
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

inputs = keras.Input(shape=(200, 250, 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)

# Use ReLU activation instead of sigmoid
x = layers.Dense(1024, activation='sigmoid')(x)

# Add dropout for regularization
x = layers.Dropout(0.5)(x)

# Adjust the number of units in the dense layer
x = layers.Dense(512, activation='sigmoid')(x)

# Add another dropout layer
x = layers.Dropout(0.5)(x)

outputs = layers.Dense(15, activation='softmax')(x)

model = keras.Model(inputs, outputs)

# Adjust learning rate and use learning rate schedule if needed
optimizer = keras.optimizers.Adam(learning_rate=0.0001)

model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])


94765736/94765736 [==============================] - 1s 0us/step


In [9]:


# Fit the model
# Fit the model
model.fit(train_images, train_labels, epochs=15, batch_size=64)


Epoch 1/15
52/52 [==============================] - 26s 158ms/step - loss: 2.3919 - accuracy: 0.2566
Epoch 2/15
52/52 [==============================] - 8s 145ms/step - loss: 1.2372 - accuracy: 0.6739
Epoch 3/15
52/52 [==============================] - 8s 149ms/step - loss: 0.5791 - accuracy: 0.8711
Epoch 4/15
52/52 [==============================] - 8s 146ms/step - loss: 0.2998 - accuracy: 0.9381
Epoch 5/15
52/52 [==============================] - 8s 148ms/step - loss: 0.1859 - accuracy: 0.9637
Epoch 6/15
52/52 [==============================] - 8s 146ms/step - loss: 0.1328 - accuracy: 0.9729
Epoch 7/15
52/52 [==============================] - 8s 148ms/step - loss: 0.0902 - accuracy: 0.9854
Epoch 8/15
52/52 [==============================] - 8s 146ms/step - loss: 0.0814 - accuracy: 0.9826
Epoch 9/15
52/52 [==============================] - 8s 147ms/step - loss: 0.0652 - accuracy: 0.9878
Epoch 10/15
52/52 [==============================] - 8s 146ms/step - loss: 0.0516 - accuracy: 0.991

In [10]:
path1 = '/kaggle/working/dataset/test'
key_value= dict()
for i in os.listdir(path1):
    key_value[i] = os.listdir(path1+'/'+i)
    
df2 = []
for key in key_value.keys():
    for j in key_value[key]:
        df2.append([key,path1+'/'+key+'/'+j])
df2 = pd.DataFrame(df2,columns=('label','path'))

import cv2
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

images = []
labels = []

for path, label in zip(df2['path'], df2['label']):
    try:
        img = cv2.imread(path)
        img = cv2.resize(img, (250, 200))
        #img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
        #img = img.resize((300, 300 * img.shape[1] // img.shape[0]), Image.ANTIALIAS)
        if img is not None:
            images.append(img)
            labels.append(label)
        else:
            print(f"Image at path {path} is None")
    except Exception as e:
        print(f"Error reading or resizing image at path: {path}")
        print(f"Error message: {str(e)}")

test_images = np.array(images, dtype='int')
labels = np.array(labels)

le = preprocessing.LabelEncoder()
test_labels = le.fit_transform(labels)



Error reading or resizing image at path: /kaggle/working/dataset/test/Western Corn Rootworms/Image_27.gif
Error message: OpenCV(4.7.0) /io/opencv/modules/imgproc/src/resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Error reading or resizing image at path: /kaggle/working/dataset/test/Fall Armyworms/Image_81.gif
Error message: OpenCV(4.7.0) /io/opencv/modules/imgproc/src/resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Error reading or resizing image at path: /kaggle/working/dataset/test/Fall Armyworms/Image_40.gif
Error message: OpenCV(4.7.0) /io/opencv/modules/imgproc/src/resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Error reading or resizing image at path: /kaggle/working/dataset/test/Armyworms/Image_49.gif
Error message: OpenCV(4.7.0) /io/opencv/modules/imgproc/src/resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

Error reading or resizing

In [11]:
from sklearn.metrics import accuracy_score

# Make predictions on the test data
predictions = model.predict(test_images)

# Convert the predictions to the class labels
predicted_labels = np.argmax(predictions, axis=1)

# Compute the accuracy score
accuracy = accuracy_score(test_labels, predicted_labels)

print('Accuracy:', accuracy)

15/15 [==============================] - 3s 104ms/step
Accuracy: 0.7217573221757322


In [12]:
from sklearn.metrics import precision_score

precision = precision_score(test_labels, predicted_labels, average='micro')

print('Precision:', precision*100)

Precision: 72.17573221757321


In [13]:
model.save('/kaggle/working/model.h5')